In [ ]:
import cv2
import numpy as np
import os
from pathlib import Path
import shutil

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


In [4]:

# Define paths
BASE_DIR = Path("/content/drive/MyDrive/Change detection")
INPUT_DIR = BASE_DIR / "input-images"
OUTPUT_DIR = BASE_DIR / "task_2_output"


In [5]:

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

def get_image_pairs(input_folder):
    before_images = {}
    after_images = {}

    for file in os.listdir(input_folder):
        if file.endswith(".jpg"):
            name = file.replace("~2", "")
            if "~2" in file:
                after_images[name] = file
            else:
                before_images[name] = file

    pairs = []
    for key in before_images:
        if key in after_images:
            pairs.append((before_images[key], after_images[key]))
    return pairs

def highlight_differences(before_path, after_path, output_path):
    img1 = cv2.imread(str(before_path))
    img2 = cv2.imread(str(after_path))

    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    diff = cv2.absdiff(gray1, gray2)
    _, thresh = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)
    kernel = np.ones((5, 5), np.uint8)
    cleaned = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    contours, _ = cv2.findContours(cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    result_img = img2.copy()
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if w * h > 500:
            cv2.rectangle(result_img, (x, y), (x + w, y + h), (0, 0, 255), 2)

    cv2.imwrite(str(output_path), result_img)

def process_images():
    image_pairs = get_image_pairs(INPUT_DIR)

    for before_name, after_name in image_pairs:
        before_path = INPUT_DIR / before_name
        after_path = INPUT_DIR / after_name

        # Save before image (X.jpg)
        shutil.copy(str(before_path), OUTPUT_DIR / before_name)

        # Save annotated after image as X~3.jpg
        output_annotated = OUTPUT_DIR / after_name.replace("~2", "~3")
        highlight_differences(before_path, after_path, output_annotated)

        print(f"✓ Processed: {before_name} & {after_name} → {output_annotated.name}")

process_images()

✓ Processed: 16.jpg & 16~2.jpg → 16~3.jpg
✓ Processed: 13.jpg & 13~2.jpg → 13~3.jpg
✓ Processed: 12.jpg & 12~2.jpg → 12~3.jpg
✓ Processed: 11.jpg & 11~2.jpg → 11~3.jpg
✓ Processed: 3.jpg & 3~2.jpg → 3~3.jpg
✓ Processed: 18.jpg & 18~2.jpg → 18~3.jpg
✓ Processed: 5.jpg & 5~2.jpg → 5~3.jpg
✓ Processed: 6.jpg & 6~2.jpg → 6~3.jpg
✓ Processed: 15.jpg & 15~2.jpg → 15~3.jpg
✓ Processed: 17.jpg & 17~2.jpg → 17~3.jpg
✓ Processed: 10.jpg & 10~2.jpg → 10~3.jpg
✓ Processed: 1.jpg & 1~2.jpg → 1~3.jpg
✓ Processed: 2.jpg & 2~2.jpg → 2~3.jpg
✓ Processed: 9.jpg & 9~2.jpg → 9~3.jpg
✓ Processed: 7.jpg & 7~2.jpg → 7~3.jpg
